In [6]:
import numpy as np
from scipy.sparse import kron, identity
from scipy.linalg import ishermitian
from scipy.sparse.linalg import eigsh
from collections import namedtuple

Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

def is_valid_block(block):
    for op in block.operator_dict.values():
        if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
            return False
    return True

is_valid_enlarged_block = is_valid_block

model_d = 2  # single-site basis size

Sx1 = np.array([[0, 1], [1, 0]], dtype='d')  # single-site S^x
Sy1 = np.array([[0, -1j], [1j, 0]], dtype=complex) # single-site S^z
Sz1 = np.array([[1, 0], [0, -1]], dtype='d')  # single-site S^z
Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+

pauli_matrices = [np.eye(2), Sx1, Sy1, Sz1]

H1 = np.array([[0, 0], [0, 0]], dtype='d')  # single-site portion of H is zero


def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of H
    J = 1.
    Jz = 0.7
    return (
        (J / 2) * (kron(Sp1, Sp2.conjugate().transpose()) + kron(Sp1.conjugate().transpose(), Sp2)) +
        Jz * kron(Sz1, Sz2)
    )

initial_block = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "conn_Sz": Sz1,
    "conn_Sp": Sp1,
})

def enlarge_block(block):
    mblock = block.basis_size
    o = block.operator_dict

    enlarged_operator_dict = {
        "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1),
        "conn_Sz": kron(identity(mblock), Sz1),
        "conn_Sp": kron(identity(mblock), Sp1),
    }

    return EnlargedBlock(length=(block.length + 1),
                         basis_size=(block.basis_size * model_d),
                         operator_dict=enlarged_operator_dict)

def rotate_and_truncate(operator, transformation_matrix):
    return transformation_matrix.conjugate().transpose().dot(operator.dot(transformation_matrix))

def single_dmrg_step(sys, env, m):
    assert is_valid_block(sys)
    assert is_valid_block(env)

    sys_enl = enlarge_block(sys)
    if sys is env:
        env_enl = sys_enl
    else:
        env_enl = enlarge_block(env)

    assert is_valid_enlarged_block(sys_enl)
    assert is_valid_enlarged_block(env_enl)

    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.operator_dict
    env_enl_op = env_enl.operator_dict
    superblock_hamiltonian = kron(sys_enl_op["H"], identity(m_env_enl)) + kron(identity(m_sys_enl), env_enl_op["H"]) + \
                             H2(sys_enl_op["conn_Sz"], sys_enl_op["conn_Sp"], env_enl_op["conn_Sz"], env_enl_op["conn_Sp"])

    (energy,), psi0 = eigsh(superblock_hamiltonian, k=1, which="SA")

    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0, psi0.conjugate().transpose())

    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])

    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    #print("truncation error:", truncation_error)

    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return newblock, energy, psi0

def graphic(sys_block, env_block, sys_label="l"):
    assert sys_label in ("l", "r")
    graphic = ("=" * sys_block.length) + "**" + ("-" * env_block.length)
    if sys_label == "r":
        graphic = graphic[::-1]
    return graphic

def finite_system_algorithm(L, m_warmup, m_sweep_list):
    assert L % 2 == 0

    block_disk = {}

    block = initial_block
    block_disk["l", block.length] = block
    block_disk["r", block.length] = block
    while 2 * block.length < L:
        #print(graphic(block, block))
        block, energy, psi = single_dmrg_step(block, block, m=m_warmup)
        #print("E/L =", energy / (block.length * 2))
        block_disk["l", block.length] = block
        block_disk["r", block.length] = block

    sys_label, env_label = "l", "r"
    sys_block = block; del block
    for m in m_sweep_list:
        while True:
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

           # print(graphic(sys_block, env_block, sys_label))
            sys_block, energy, psi = single_dmrg_step(sys_block, env_block, m=m)

           # print("E/L =", energy / L)

            block_disk[sys_label, sys_block.length] = sys_block

            if sys_label == "l" and 2 * sys_block.length == L:
                return psi

def partial_trace(rho, dims, trace_sites):
    """
    Takes the partial trace over the subsystems defined in 'trace_sites'.
    rho: a matrix (density matrix)
    dims: a list containing the dimension of each subsystem
    trace_sites: a list of indices of the subsystems to be traced out
    (We assume that each subsystem is square)                 
    """
    dims = np.array(dims)
    n_sites = len(dims)
    total_dims = np.prod(dims)
    if rho.shape != (total_dims, total_dims):
        raise ValueError("The shape of rho does not match the product of dims.")
    
    # Determine which sites to keep
    keep_sites = sorted(set(range(n_sites)) - set(trace_sites))
    
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(np.concatenate((dims, dims)))
    
    # Permute the tensor to group the subsystems we want to trace at the end
    perm = keep_sites + trace_sites + [i + n_sites for i in keep_sites] + [i + n_sites for i in trace_sites]
    reshaped_rho = np.transpose(reshaped_rho, perm)
    
    # Compute the dimensions after permutation
    keep_dim = np.prod(dims[keep_sites])
    trace_dim = np.prod(dims[trace_sites])
    
    # Reshape the permuted tensor and perform the partial trace
    reshaped_rho = reshaped_rho.reshape((keep_dim, trace_dim, keep_dim, trace_dim))
    reduced_rho = np.trace(reshaped_rho, axis1=1, axis2=3)
    
    # Normalize the reduced density matrix to have trace 1
    reduced_rho /= np.trace(reduced_rho)
    return reduced_rho
    

#def partial_transpose(rho, dims, subsystem):
 #   dims = np.array(dims)
    #dim1 = np.prod([dims[i] for i in range(len(dims)) if i not in subsystem])
    #dim1 = np.prod([dims[i] for i in subsystem])
    #dim2 = np.prod([dims[i] for i in subsystem])
  #  dim1 = 2
   # dim2 = np.prod(n1)//2 
    
    #rho = rho.reshape([dim1, dim2, dim1, dim2]).swapaxes(0, 2).reshape(dim1 * dim2, dim1 * dim2)
    #rho = rho.transpose(0, 3, 2, 1)
    #rho = rho.reshape([dim1 * dim2, dim1 * dim2])
    
    #rho = p.reshape(2, 2, 2, 2).swapaxes(0, 2).reshape(4, 4)
    
    #return rho
    
# Function to modify tensor product based on focus (A, B, C, D)
def modify_pauli_tensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'A' :
        if i == 2:
            matrices[0] = -Sy1 # Replace Pauli Y on A
    elif focus == 'B':
        if j == 2:
            matrices[1] = -Sy1   # Replace Pauli Y on B
    elif focus == 'C':
        if k == 2:
            matrices[2] = -Sy1   # Replace Pauli Y on C
    elif focus == 'D' :
        if l == 2:
            matrices[3] = -Sy1   # Replace Pauli Y on D
    
    
    return np.kron(np.kron(np.kron(matrices[0], matrices[1]), matrices[2]), matrices[3])


def modify_pauli_dtensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'AB':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if j == 2:
            matrices[1] = -Sy1   # Replace Pauli Y on B
    elif focus == 'AC':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if k == 2:
            matrices[2] = -Sy1   # Replace Pauli Y on C
    elif focus == 'AD':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if l == 2:
            matrices[3] = -Sy1   # Replace Pauli Y on D
    
    return np.kron(np.kron(np.kron(matrices[0], matrices[1]), matrices[2]), matrices[3])



def print_eigenvalues(tensor_sum, label):
    tensor_real = np.real(tensor_sum)  # Convert to real matrix (no need for .full())
    eigenvalues = np.linalg.eigvalsh(tensor_real)  # Compute eigenvalues
    print(f"Eigenvalues for {label}: {eigenvalues}")



def logarithmic_negativity(rho):
    """
    Compute the logarithmic negativity of the density matrix.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The logarithmic negativity of the density matrix.
    """
    # Perform partial transpose
    eigenvalues = np.linalg.eigvalsh(rho)
    
    # Compute negativity (sum of absolute values of negative eigenvalues)
    negativity = np.sum(np.abs(eigenvalues[eigenvalues < 0]))
    
    # Compute logarithmic negativity
    log_negativity = np.log(2*negativity + 1)  # Adding small value to avoid log(0)
    
    return log_negativity


#def logarithmic_negativity(rho, dims, subsystem):
    #rho_pt = partial_transpose(rho, dims, subsystem)
   # eigenvalues = np.linalg.eigvals(rho_pt)
   # negativity = sum(np.abs(eigenvalues) - eigenvalues) / 2
   # log_negativity = np.log(2*negativity + 1)
    
    #return log_negativity



if __name__ == "__main__":
    np.set_printoptions(precision=10, suppress=True, threshold=10000, linewidth=300)

    L = 16
    dims = [model_d] * L
    m_warmup = 15
    m_sweep_list = [128]
    dims1 =np.array(dims)
    total_dims = np.prod(dims1)
    
  
    sites1 = [2,3,4,5,6,7,8,9,10,11,12,13] 
  
    n1=np.delete(dims, sites1)
    
    # Initialize tensor product sums to handle complex values
    sum_tensor_products_A = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_B = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_C = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_D = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_AB = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_AC = np.zeros((2**4, 2**4), dtype='complex128')
    sum_tensor_products_AD = np.zeros((2**4, 2**4), dtype='complex128')

    psi = finite_system_algorithm(L, m_warmup, m_sweep_list)
    psi = psi.reshape(dims)
    rho = np.outer(psi, psi.conj()).reshape((total_dims, total_dims))
    
     # Specify the sites of interest (0-based indexing)
    rho_sites= partial_trace(rho, dims1, sites1)
    #rho_sitesb= partial_trace(rho, dims1, sites2)
    #rho_sites=np.kron(rho_sitesa,rho_sitesb)

        # Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    # Calculate the tensor products with the focus on each qubit A, B, C, D
                    tensor_product_A = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'A')) * modify_pauli_tensor(i, j, k, l, 'A') 
                   
                    # Add the modified tensor products to their respective sums
                sum_tensor_products_A += tensor_product_A
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    # Calculate the tensor products with the focus on each qubit A, B, C, D            
                    tensor_product_B = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'B')) * modify_pauli_tensor(i, j, k, l, 'B')              
                    
                sum_tensor_products_B += tensor_product_B
                    
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    # Calculate the tensor products with the focus on each qubit A, B, C, D
                    tensor_product_C = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'C')) * modify_pauli_tensor(i, j, k, l, 'C') 
                    
                sum_tensor_products_C += tensor_product_C
                    
                    
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    # Calculate the tensor products with the focus on each qubit A, B, C, D
                    tensor_product_D = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'D')) * modify_pauli_tensor(i, j, k, l, 'D') 
                    
                sum_tensor_products_D += tensor_product_D
                    
        
     # Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    # Calculate the tensor products with the focus on each qubit A, B, C, D
                    tensor_dproduct_AB = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AB')) * modify_pauli_dtensor(i, j, k, l, 'AB') 
                    tensor_dproduct_AC = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AC')) * modify_pauli_dtensor(i, j, k, l, 'AC') 
                    tensor_dproduct_AD = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AD')) * modify_pauli_dtensor(i, j, k, l, 'AD') 
                    
                    # Add the modified tensor products to their respective sums
                sum_tensor_products_AB += tensor_dproduct_AB
                sum_tensor_products_AC += tensor_dproduct_AC
                sum_tensor_products_AD += tensor_dproduct_AD
                    
    
    v1=logarithmic_negativity(sum_tensor_products_A)
    v2=logarithmic_negativity(sum_tensor_products_B)
    v3=logarithmic_negativity(sum_tensor_products_C)
    v4=logarithmic_negativity(sum_tensor_products_D)
    v5=logarithmic_negativity(sum_tensor_products_AB)
    v6=logarithmic_negativity(sum_tensor_products_AC)
    v7=logarithmic_negativity(sum_tensor_products_AD)
    v=(v1*v2*v3*v4*v5*v6*v7)**(1/7)
    
    
    def is_symmetric(matrix, tol=1e-8):
        return np.allclose(matrix, matrix.conjugate().T, atol=tol)

             
    #print(rho_sites)
    #print(rho_sites.shape)
    print(ishermitian(rho))  # Should return True
    print(ishermitian(rho_sites))
    print(is_symmetric(rho_sites))
    #print("Logarithmic Negativity for sites {}:".format(n1), values)
    # Function to compute and print eigenvalues

   # Compute and print eigenvalues for each tensor product sum
    print_eigenvalues(sum_tensor_products_A, 'sum_tensor_products_A')
    print_eigenvalues(sum_tensor_products_B, 'sum_tensor_products_B')
    print_eigenvalues(sum_tensor_products_C, 'sum_tensor_products_C')
    print_eigenvalues(sum_tensor_products_D, 'sum_tensor_products_D')
    print_eigenvalues(sum_tensor_products_AB, 'sum_tensor_products_AB')
    print_eigenvalues(sum_tensor_products_AC, 'sum_tensor_products_AC')
    print_eigenvalues(sum_tensor_products_AD, 'sum_tensor_products_AD')
    print('Negativity w.r.t. A :', v1)
    print('Negativity w.r.t. B :', v2)
    print('Negativity w.r.t. C :', v3)
    print('Negativity w.r.t. D :', v4)
    print('Negativity w.r.t. AB :', v5)
    print('Negativity w.r.t. AC :', v6)
    print('Negativity w.r.t. AD :', v7)
    print('Negativity :', v)

    
    #log_neg = logarithmic_negativity(rho_sites, dims, list(range(len(dims)-len(sites))))
   # print("Logarithmic Negativity for sites {}:".format(sites), log_neg)


True
True
True
Eigenvalues for sum_tensor_products_A: [-0.2223062619 -0.2223062619 -0.           -0.           -0.           -0.           -0.           -0.            0.            0.            0.            0.            0.            0.            0.2223062619  0.2223062619]
Eigenvalues for sum_tensor_products_B: [-0.2223062619 -0.2223062619 -0.           -0.           -0.           -0.           -0.           -0.            0.            0.            0.            0.            0.            0.            0.2223062619  0.2223062619]
Eigenvalues for sum_tensor_products_C: [-0.2223062619 -0.2223062619 -0.           -0.           -0.           -0.           -0.           -0.            0.            0.            0.            0.            0.            0.            0.2223062619  0.2223062619]
Eigenvalues for sum_tensor_products_D: [-0.2223062619 -0.2223062619 -0.           -0.           -0.           -0.           -0.           -0.            0.            0.            0.       

In [ ]:
for i in range(3):
    print (i)

In [ ]:
import numpy as np
from scipy.sparse import kron, identity
from scipy.linalg import ishermitian
from scipy.sparse.linalg import eigsh
from collections import namedtuple


def partial_trace(rho, dims, trace_sites):
    """
    Takes the partial trace over the subsystems defined in 'trace_sites'.
    rho: a matrix (density matrix)
    dims: a list containing the dimension of each subsystem
    trace_sites: a list of indices of the subsystems to be traced out
    (We assume that each subsystem is square)                 
    """
    dims = np.array(dims)
    n_sites = len(dims)
    total_dims = np.prod(dims)
    if rho.shape != (total_dims, total_dims):
        raise ValueError("The shape of rho does not match the product of dims.")
    
    # Determine which sites to keep
    keep_sites = sorted(set(range(n_sites)) - set(trace_sites))
    
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(np.concatenate((dims, dims)))
    
    # Permute the tensor to group the subsystems we want to trace at the end
    perm = keep_sites + trace_sites + [i + n_sites for i in keep_sites] + [i + n_sites for i in trace_sites]
    reshaped_rho = np.transpose(reshaped_rho, perm)
    
    # Compute the dimensions after permutation
    keep_dim = np.prod(dims[keep_sites])
    trace_dim = np.prod(dims[trace_sites])
    
    # Reshape the permuted tensor and perform the partial trace
    reshaped_rho = reshaped_rho.reshape((keep_dim, trace_dim, keep_dim, trace_dim))
    reduced_rho = np.trace(reshaped_rho, axis1=1, axis2=3)
    
    # Normalize the reduced density matrix to have trace 1
    reduced_rho /= np.trace(reduced_rho)
    return reduced_rho
    

def partial_transpose(rho, dims, subsystem):
    dims = np.array(dims)
    dim1 = 2
    dim2 = np.prod(n1)//2 
    
    rho = rho.reshape([dim1, dim2, dim1, dim2]).swapaxes(0, 2).reshape(dim1 * dim2, dim1 * dim2)
    #rho = rho.transpose(0, 3, 2, 1)
    #rho = rho.reshape([dim1 * dim2, dim1 * dim2])
    
    #rho = p.reshape(2, 2, 2, 2).swapaxes(0, 2).reshape(4, 4)
    
    return rho

def logarithmic_negativity(rho, dims, subsystem):
    rho_pt = partial_transpose(rho, dims, subsystem)
    eigenvalues = np.linalg.eigvals(rho_pt)
    negativity = sum(np.abs(eigenvalues) - eigenvalues) / 2
    log_negativity = np.log(2*negativity + 1)
    
    return log_negativity



L = 16
dims = [2] * 4
dims1 =np.array(dims)
total_dims = np.prod(dims1)
sites = [1] 
n1=np.delete(dims, sites)
# Define the GHZ state for 4 qubits
ghz_state = (np.kron(np.kron(np.kron([1, 0], [1, 0]), [1, 0]), [1, 0]) +
             np.kron(np.kron(np.kron([0, 1], [0, 1]), [0, 1]), [0, 1])) / np.sqrt(2)

# Density matrix of the GHZ state
rho = np.outer(ghz_state, ghz_state.conj())

# Sanity check: ensure it's a 16x16 matrix
#assert rho_ghz.shape == (16, 16), "Density matrix should be 16x16 for 4 qubits"

rho_sites= partial_trace(rho, dims1, sites)
#rho_sites=partial_transpose(rho_sites, n1, n1)
values=logarithmic_negativity(rho_sites, n1, n1)
    
print(rho_sites)
print(rho_sites.shape)
print(ishermitian(rho_sites))
print("Logarithmic Negativity for sites {}:".format(n1), values)
    

# Print the density matrix for the GHZ state
#print("Density matrix for the 4-qubit GHZ state:\n", rho_ghz)



In [ ]:
import numpy as np

# Define the GHZ state for 4 qubits (dimension 16)
ghz_state = (np.kron(np.kron(np.kron([1, 0], [1, 0]), [1, 0]), [1, 0]) +
             np.kron(np.kron(np.kron([0, 1], [0, 1]), [0, 1]), [0, 1])) / np.sqrt(2)

# Density matrix of the GHZ state
rho_ghz = np.outer(ghz_state, ghz_state.conj())

def partial_transpose(rho, dims, subsystem):
    """
    Perform the partial transpose with respect to the given subsystem.
    rho: The density matrix.
    dims: The dimensions of the subsystems.
    subsystem: The index of the subsystem to be partially transposed.
    """
    n = len(dims)  # Number of subsystems
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(dims + dims)
    
    # Swap axes to perform partial transpose on the specified subsystem
    reshaped_rho = np.swapaxes(reshaped_rho, subsystem, subsystem + n)
    
    # Reshape back into a matrix
    reshaped_rho = reshaped_rho.reshape(np.prod(dims), np.prod(dims))
    
    return reshaped_rho

# Perform the partial transpose on the first qubit (subsystem 0) of the 4-qubit GHZ state
partial_transposed_rho = partial_transpose(rho_ghz, [2, 2, 2, 2], 0)

# Compute the eigenvalues of the partially transposed matrix
eigenvalues = np.linalg.eigvalsh(partial_transposed_rho)

# Check for negative eigenvalues (sign of entanglement)
print("Eigenvalues of the partial transpose of the GHZ state:", eigenvalues)
if np.any(eigenvalues < 0):
    print("The partial transpose has negative eigenvalues, indicating entanglement.")
else:
    print("The partial transpose has no negative eigenvalues, indicating separability.")


In [ ]:


def partial_trace(rho, dims, trace_sites):
    """
    Perform partial trace over the subsystems defined in 'trace_sites'.
    
    Parameters:
    rho: The full density matrix.
    dims: A list containing the dimension of each subsystem.
    trace_sites: The indices of the subsystems to be traced out.
    
    Returns:
    The reduced density matrix after tracing out the subsystems.
    """
    dims = np.array(dims)
    n_sites = len(dims)
    total_dims = np.prod(dims)
    if rho.shape != (total_dims, total_dims):
        raise ValueError("The shape of rho does not match the product of dims.")
    
    # Determine which sites to keep
    keep_sites = sorted(set(range(n_sites)) - set(trace_sites))
    
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(np.concatenate((dims, dims)))
    
    # Permute the tensor to group the subsystems we want to trace at the end
    perm = keep_sites + trace_sites + [i + n_sites for i in keep_sites] + [i + n_sites for i in trace_sites]
    reshaped_rho = np.transpose(reshaped_rho, perm)
    
    # Compute the dimensions after permutation
    keep_dim = np.prod(dims[keep_sites])
    trace_dim = np.prod(dims[trace_sites])
    
    # Reshape the permuted tensor and perform the partial trace
    reshaped_rho = reshaped_rho.reshape((keep_dim, trace_dim, keep_dim, trace_dim))
    reduced_rho = np.trace(reshaped_rho, axis1=1, axis2=3)
    
    # Normalize the reduced density matrix to have trace 1
    reduced_rho /= np.trace(reduced_rho)
    
    return reduced_rho

def partial_transpose(rho, dims, subsystem):
    """
    Perform the partial transpose with respect to the given subsystem.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The partially transposed density matrix.
    """
    n = len(dims)  # Number of subsystems
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(dims + dims)
    
    # Swap axes to perform partial transpose on the specified subsystem
    reshaped_rho = np.swapaxes(reshaped_rho, subsystem, subsystem + n)
    
    # Reshape back into a matrix
    reshaped_rho = reshaped_rho.reshape(np.prod(dims), np.prod(dims))
    
    return reshaped_rho

# Define the GHZ state for 4 qubits (dimension 16)
ghz_state = (np.kron(np.kron(np.kron([1, 0], [1, 0]), [1, 0]), [1, 0]) +
             np.kron(np.kron(np.kron([0, 1], [0, 1]), [0, 1]), [0, 1])) / np.sqrt(2)

# Density matrix of the GHZ state
rho_ghz = np.outer(ghz_state, ghz_state.conj())

# Step 1: Partial trace over one of the qubits (e.g., trace out the 3rd qubit)
# dims = [2, 2, 2, 2] for a 4-qubit system
reduced_rho = partial_trace(rho_ghz, [2, 2, 2, 2], [2])
#reduced_rho = partial_trace(reduced_rhoa, [2, 2, 2], [0])
# Step 2: Partial transpose on the reduced system (let's transpose the first qubit after tracing out)
partially_transposed_rho = partial_transpose(reduced_rho, [2, 2, 2], 0)

# Step 3: Calculate eigenvalues to check for negative values
eigenvalues = np.linalg.eigvalsh(partially_transposed_rho)

# Check for negative eigenvalues (indicating entanglement)
print("Eigenvalues of the partial transpose of the GHZ state after partial trace:", eigenvalues)
if np.any(eigenvalues < 0):
    print("The partial transpose has negative eigenvalues, indicating entanglement.")
else:
    print("The partial transpose has no negative eigenvalues, indicating separability.")


In [ ]:
import numpy as np

def partial_trace(rho, dims, trace_sites):
    """
    Perform partial trace over the subsystems defined in 'trace_sites'.
    
    Parameters:
    rho: The full density matrix.
    dims: A list containing the dimension of each subsystem.
    trace_sites: The indices of the subsystems to be traced out.
    
    Returns:
    The reduced density matrix after tracing out the subsystems.
    """
    dims = np.array(dims)
    n_sites = len(dims)
    total_dims = np.prod(dims)
    if rho.shape != (total_dims, total_dims):
        raise ValueError("The shape of rho does not match the product of dims.")
    
    # Determine which sites to keep
    keep_sites = sorted(set(range(n_sites)) - set(trace_sites))
    
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(np.concatenate((dims, dims)))
    
    # Permute the tensor to group the subsystems we want to trace at the end
    perm = keep_sites + trace_sites + [i + n_sites for i in keep_sites] + [i + n_sites for i in trace_sites]
    reshaped_rho = np.transpose(reshaped_rho, perm)
    
    # Compute the dimensions after permutation
    keep_dim = np.prod(dims[keep_sites])
    trace_dim = np.prod(dims[trace_sites])
    
    # Reshape the permuted tensor and perform the partial trace
    reshaped_rho = reshaped_rho.reshape((keep_dim, trace_dim, keep_dim, trace_dim))
    reduced_rho = np.trace(reshaped_rho, axis1=1, axis2=3)
    
    # Normalize the reduced density matrix to have trace 1
    reduced_rho /= np.trace(reduced_rho)
    
    return reduced_rho

def partial_transpose(rho, dims, subsystem):
    """
    Perform the partial transpose with respect to the given subsystem.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The partially transposed density matrix.
    """
    n = len(dims)  # Number of subsystems
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(dims + dims)
    
    # Swap axes to perform partial transpose on the specified subsystem
    reshaped_rho = np.swapaxes(reshaped_rho, subsystem, subsystem + n)
    
    # Reshape back into a matrix
    reshaped_rho = reshaped_rho.reshape(np.prod(dims), np.prod(dims))
    
    return reshaped_rho

# Define the GHZ state for 4 qubits (dimension 16)
ghz_state = (np.kron(np.kron(np.kron([1, 0], [1, 0]), [1, 0]), [1, 0]) +
             np.kron(np.kron(np.kron([0, 1], [0, 1]), [0, 1]), [0, 1])) / np.sqrt(2)

# Density matrix of the GHZ state
rho_ghz = np.outer(ghz_state, ghz_state.conj())

# Step 1: Partial trace over two of the qubits (e.g., trace out the 2nd and 3rd qubits)
# dims = [2, 2, 2, 2] for a 4-qubit system
reduced_rho = partial_trace(rho_ghz, [2, 2, 2, 2], [0, 1])

# Step 2: Partial transpose on the reduced system (let's transpose the first qubit after tracing out)
partially_transposed_rho = partial_transpose(reduced_rho, [2, 2], 0)

# Step 3: Calculate eigenvalues to check for negative values
eigenvalues = np.linalg.eigvalsh(partially_transposed_rho)

# Check for negative eigenvalues (indicating entanglement)
print("Eigenvalues of the partial transpose of the GHZ state after partial trace over two qubits:", eigenvalues)
if np.any(eigenvalues < 0):
    print("The partial transpose has negative eigenvalues, indicating entanglement.")
else:
    print("The partial transpose has no negative eigenvalues, indicating separability.")


In [ ]:
import numpy as np

def partial_trace(rho, dims, trace_sites):
    """
    Perform partial trace over the subsystems defined in 'trace_sites'.
    
    Parameters:
    rho: The full density matrix.
    dims: A list containing the dimension of each subsystem.
    trace_sites: The indices of the subsystems to be traced out.
    
    Returns:
    The reduced density matrix after tracing out the subsystems.
    """
    dims = np.array(dims)
    n_sites = len(dims)
    total_dims = np.prod(dims)
    if rho.shape != (total_dims, total_dims):
        raise ValueError("The shape of rho does not match the product of dims.")
    
    # Determine which sites to keep
    keep_sites = sorted(set(range(n_sites)) - set(trace_sites))
    
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(np.concatenate((dims, dims)))
    
    # Permute the tensor to group the subsystems we want to trace at the end
    perm = keep_sites + trace_sites + [i + n_sites for i in keep_sites] + [i + n_sites for i in trace_sites]
    reshaped_rho = np.transpose(reshaped_rho, perm)
    
    # Compute the dimensions after permutation
    keep_dim = np.prod(dims[keep_sites])
    trace_dim = np.prod(dims[trace_sites])
    
    # Reshape the permuted tensor and perform the partial trace
    reshaped_rho = reshaped_rho.reshape((keep_dim, trace_dim, keep_dim, trace_dim))
    reduced_rho = np.trace(reshaped_rho, axis1=1, axis2=3)
    
    # Normalize the reduced density matrix to have trace 1
    reduced_rho /= np.trace(reduced_rho)
    
    return reduced_rho

def partial_transpose(rho, dims, subsystem):
    """
    Perform the partial transpose with respect to the given subsystem.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The partially transposed density matrix.
    """
    n = len(dims)  # Number of subsystems
    # Reshape rho into a tensor with shape [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    reshaped_rho = rho.reshape(dims + dims)
    
    # Swap axes to perform partial transpose on the specified subsystem
    reshaped_rho = np.swapaxes(reshaped_rho, subsystem, subsystem + n)
    
    # Reshape back into a matrix
    reshaped_rho = reshaped_rho.reshape(np.prod(dims), np.prod(dims))
    
    return reshaped_rho

def logarithmic_negativity(rho, dims, subsystem):
    """
    Compute the logarithmic negativity of the density matrix.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The logarithmic negativity of the density matrix.
    """
    # Perform partial transpose
    partial_transposed_rho = partial_transpose(rho, dims, subsystem)
    
    # Compute eigenvalues of the partially transposed matrix
    eigenvalues = np.linalg.eigvalsh(partial_transposed_rho)
    
    # Compute negativity (sum of absolute values of negative eigenvalues)
    negativity = np.sum(np.abs(eigenvalues[eigenvalues < 0]))
    
    # Compute logarithmic negativity
    log_negativity = np.log(2*negativity + 1)  # Adding small value to avoid log(0)
    
    return log_negativity

# Define the GHZ state for 4 qubits (dimension 16)
ghz_state = (np.kron(np.kron(np.kron([1, 0], [1, 0]), [1, 0]), [1, 0]) +
             np.kron(np.kron(np.kron([0, 1], [0, 1]), [0, 1]), [0, 1])) / np.sqrt(2)

# Density matrix of the GHZ state
rho_ghz = np.outer(ghz_state, ghz_state.conj())

# Step 1: Partial trace over two of the qubits (e.g., trace out the 2nd and 3rd qubits)
# dims = [2, 2, 2, 2] for a 4-qubit system
reduced_rho = partial_trace(rho_ghz, [2, 2, 2, 2], [ 2])

# Step 2: Calculate logarithmic negativity of the reduced state
# We assume we want to check entanglement of the remaining two-qubit state,
# so we partially transpose one of the qubits in the remaining 2-qubit system.
log_neg = logarithmic_negativity(reduced_rho, [2, 2, 2], 1)

# Print results
print("Logarithmic Negativity of the reduced density matrix after partial trace:", log_neg)


In [ ]:
import numpy as np
from qutip import *

# Step 1: Define the 4-partite GHZ state
# GHZ state: |GHZ_4> = (|0000> + |1111>)/sqrt(2)

# Basis states |0000> and |1111>
state_0000 = tensor(basis(2, 0), basis(2, 0), basis(2, 0), basis(2, 0))  # |0000>
state_1111 = tensor(basis(2, 1), basis(2, 1), basis(2, 1), basis(2, 1))  # |1111>

# GHZ state as a superposition of |0000> and |1111>
ghz_state = (state_0000 + state_1111).unit()  # Normalize the state

# Step 2: Convert the GHZ state to a density matrix
rho_ghz = ghz_state * ghz_state.dag()  # Density matrix of the GHZ state

# Step 3: Print the density matrix of the 4-partite GHZ state
print("Density matrix of the 4-partite GHZ state:")
print(rho_ghz)




In [ ]:
from qutip import *
import numpy as np

# Step 1: Define the Pauli matrices and the identity matrix
pauli_matrices = [qeye(2), sigmax(), sigmay(), sigmaz()]  # [I, X, Y, Z]

# Step 2: Compute the sum over the tensor products of the 4 Pauli matrices
sum_tensor_products = 0  # Initialize sum

# Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Compute the tensor product for the current combination
                tensor_product = tensor(pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l])
                # Add the tensor product to the sum
                sum_tensor_products += tensor_product

# Step 3: Print the result
print("Sum over the tensor products of the 4 Pauli matrices:")
print(sum_tensor_products)




In [ ]:
from qutip import *
import numpy as np


# Step 1: Define the 4-partite GHZ state |GHZ> = (|0000> + |1111>) / sqrt(2)
ghz_state = (tensor(basis(2, 0), basis(2, 0), basis(2, 0), basis(2, 0)) + 
             tensor(basis(2, 1), basis(2, 1), basis(2, 1), basis(2, 1))).unit()

# GHZ density matrix: rho_GHZ = |GHZ><GHZ|
ghz_density_matrix = ghz_state.proj()

# Step 2: Define the Pauli matrices (I, X, Y, Z)
pauli_matrices = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Step 3: Initialize the sum over tensor products of Pauli matrices and the trace result
sum_tensor_products = 0
trace_sum = 0

# Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Compute the tensor product for the current combination
                tensor_product = (1/16) * tensor(pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]) * \
                    (ghz_density_matrix*tensor(pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l])).tr()
                
                # Add the tensor product to the sum (no trace scaling)
                sum_tensor_products += tensor_product

# Step 4: Calculate the trace of the sum of tensor products
trace_sum = (ghz_density_matrix * sum_tensor_products).tr()

# Step 4: Print the result
#print("Sum over the tensor products of the 4 Pauli matrices, multiplied by their corresponding traces:")
#print(sum_tensor_products)


sum_tensor_real = np.real(sum_tensor_products.full())
eigenvalues = np.linalg.eigvalsh(sum_tensor_real)
eigenvalues1= np.linalg.eigvalsh(ghz_density_matrix)
#print("Sum over the tensor products of the 4 Pauli matrices, multiplied by their corresponding traces:")
print(np.sum(eigenvalues))
print(eigenvalues)
print(trace_sum)
print(sum_tensor_products.tr())



In [ ]:
from qutip import *
import numpy as np

# Step 1: Define the 4-partite GHZ state |GHZ> = (|0000> + |1111>) / sqrt(2)
ghz_state = (tensor(basis(2, 0), basis(2, 0), basis(2, 0), basis(2, 0)) + 
             tensor(basis(2, 1), basis(2, 1), basis(2, 1), basis(2, 1))).unit()

# GHZ density matrix: rho_GHZ = |GHZ><GHZ|
ghz_density_matrix = ghz_state.proj()

# Step 2: Define the Pauli matrices (I, X, Y, Z)
pauli_matrices = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Step 3: Initialize the sum over tensor products of Pauli matrices
sum_tensor_products_A = 0
sum_tensor_products_B = 0
sum_tensor_products_C = 0
sum_tensor_products_D = 0
sum_tensor_products_AB = 0
sum_tensor_products_AC = 0
sum_tensor_products_AD = 0


# Function to modify tensor product based on focus (A, B, C, D)
def modify_pauli_tensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'A' :
        if i == 2:
            matrices[0] = -Sy1 # Replace Pauli Y on A
    elif focus == 'B':
        if j == 2:
            matrices[1] = -Sy1   # Replace Pauli Y on B
    elif focus == 'C':
        if k == 2:
            matrices[2] = -Sy1   # Replace Pauli Y on C
    elif focus == 'D' :
        if l == 2:
            matrices[3] = -Sy1   # Replace Pauli Y on D
    
    return tensor(matrices[0], matrices[1], matrices[2], matrices[3])

def modify_pauli_dtensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'AB':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if j == 2:
            matrices[1] = -sigmay()  # Replace Pauli Y on B
    elif focus == 'AC':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if k == 2:
            matrices[2] = -sigmay()  # Replace Pauli Y on C
    elif focus == 'AD':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if l == 2:
            matrices[3] = -sigmay()  # Replace Pauli Y on D
    
    return tensor(matrices[0], matrices[1], matrices[2], matrices[3])


# Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Calculate the tensor products with the focus on each qubit A, B, C, D
                tensor_product_A = (1/16) * modify_pauli_tensor(i, j, k, l, 'A') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'A')).tr()
                tensor_product_B = (1/16) * modify_pauli_tensor(i, j, k, l, 'B') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'B')).tr()
                tensor_product_C = (1/16) * modify_pauli_tensor(i, j, k, l, 'C') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'C')).tr()
                tensor_product_D = (1/16) * modify_pauli_tensor(i, j, k, l, 'D') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'D')).tr()
                
                # Add the modified tensor products to their respective sums
                sum_tensor_products_A += tensor_product_A
                sum_tensor_products_B += tensor_product_B
                sum_tensor_products_C += tensor_product_C
                sum_tensor_products_D += tensor_product_D
                
                
                
                
# Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Calculate the tensor products with the focus on each qubit A, B, C, D
                tensor_dproduct_AB = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AB') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AB')).tr()
                tensor_dproduct_AC = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AC') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AC')).tr()
                tensor_dproduct_AD = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AD') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AD')).tr()
                
                # Add the modified tensor products to their respective sums
                sum_tensor_products_AB += tensor_dproduct_AB
                sum_tensor_products_AC += tensor_dproduct_AC
                sum_tensor_products_AD += tensor_dproduct_AD
                
                
                
sum_tensor_real = np.real(sum_tensor_products_AD.full())               
eigenvalues = np.linalg.eigvalsh(sum_tensor_real)
# Step 5: Compute and print the traces for each focus
trace_A = sum_tensor_products_A.tr()
trace_B = sum_tensor_products_B.tr()
trace_C = sum_tensor_products_C.tr()
trace_D = sum_tensor_products_D.tr()

# Step 5: Compute and print the traces for each focus
trace_AB = sum_tensor_products_AB.tr()
trace_AC = sum_tensor_products_AC.tr()
trace_AD = sum_tensor_products_AD.tr()

print(f"Trace for focus on A: {trace_A}")
print(f"Trace for focus on B: {trace_B}")
print(f"Trace for focus on C: {trace_C}")
print(f"Trace for focus on D: {trace_D}")

print(f"Trace for focus on AB: {trace_AB}")
print(f"Trace for focus on AC: {trace_AC}")
print(f"Trace for focus on AD: {trace_AD}")

print(eigenvalues)

# Optional: visualize the sums (real parts) for each focus


In [ ]:
from qutip import *
import numpy as np

# Step 1: Define the 4-partite GHZ state |GHZ> = (|0000> + |1111>) / sqrt(2)
ghz_state = (tensor(basis(2, 0), basis(2, 0), basis(2, 0), basis(2, 0)) + 
             tensor(basis(2, 1), basis(2, 1), basis(2, 1), basis(2, 1))).unit()

# GHZ density matrix: rho_GHZ = |GHZ><GHZ|
ghz_density_matrix = ghz_state.proj()

# Step 2: Define the Pauli matrices (I, X, Y, Z)
pauli_matrices = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Step 3: Initialize the sum over tensor products of Pauli matrices
sum_tensor_products_A = 0
sum_tensor_products_B = 0
sum_tensor_products_C = 0
sum_tensor_products_D = 0
sum_tensor_products_AB = 0
sum_tensor_products_AC = 0
sum_tensor_products_AD = 0

# Function to modify tensor product based on focus (A, B, C, D)
def modify_pauli_tensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'A' and i == 2:
        matrices[0] = -sigmay()  # Replace Pauli Y on A
    if focus == 'B' and j == 2:
        matrices[1] = -sigmay()  # Replace Pauli Y on B
    if focus == 'C' and k == 2:
        matrices[2] = -sigmay()  # Replace Pauli Y on C
    if focus == 'D' and l == 2:
        matrices[3] = -sigmay()  # Replace Pauli Y on D
    
    return tensor(matrices[0], matrices[1], matrices[2], matrices[3])

def modify_pauli_dtensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'AB':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if j == 2:
            matrices[1] = -sigmay()  # Replace Pauli Y on B
    elif focus == 'AC':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if k == 2:
            matrices[2] = -sigmay()  # Replace Pauli Y on C
    elif focus == 'AD':
        if i == 2:
            matrices[0] = -sigmay()  # Replace Pauli Y on A
        if l == 2:
            matrices[3] = -sigmay()  # Replace Pauli Y on D
    
    return tensor(matrices[0], matrices[1], matrices[2], matrices[3])



# Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Calculate the tensor products with the focus on each qubit A, B, C, D
                tensor_product_A = (1/16) * modify_pauli_tensor(i, j, k, l, 'A') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'A')).tr()
                tensor_product_B = (1/16) * modify_pauli_tensor(i, j, k, l, 'B') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'B')).tr()
                tensor_product_C = (1/16) * modify_pauli_tensor(i, j, k, l, 'C') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'C')).tr()
                tensor_product_D = (1/16) * modify_pauli_tensor(i, j, k, l, 'D') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'D')).tr()
                
                # Add the modified tensor products to their respective sums
                sum_tensor_products_A += tensor_product_A
                sum_tensor_products_B += tensor_product_B
                sum_tensor_products_C += tensor_product_C
                sum_tensor_products_D += tensor_product_D

# Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                # Calculate the tensor products with the focus on each qubit A, B, C, D
                tensor_dproduct_AB = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AB') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AB')).tr()
                tensor_dproduct_AC = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AC') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AC')).tr()
                tensor_dproduct_AD = (1/16) * modify_pauli_dtensor(i, j, k, l, 'AD') * (ghz_density_matrix * modify_pauli_tensor(i, j, k, l, 'AD')).tr()
                
                # Add the modified tensor products to their respective sums
                sum_tensor_products_AB += tensor_dproduct_AB
                sum_tensor_products_AC += tensor_dproduct_AC
                sum_tensor_products_AD += tensor_dproduct_AD


# Function to compute and print eigenvalues
def print_eigenvalues(tensor_sum, label):
    tensor_real = np.real(tensor_sum.full())  # Convert to real matrix
    eigenvalues = np.linalg.eigvalsh(tensor_real)  # Compute eigenvalues
    print(f"Eigenvalues for {label}: {eigenvalues}")

# Compute and print eigenvalues for each tensor product sum
print_eigenvalues(sum_tensor_products_A, 'sum_tensor_products_A')
print_eigenvalues(sum_tensor_products_B, 'sum_tensor_products_B')
print_eigenvalues(sum_tensor_products_C, 'sum_tensor_products_C')
print_eigenvalues(sum_tensor_products_D, 'sum_tensor_products_D')
print_eigenvalues(sum_tensor_products_AB, 'sum_tensor_products_AB')
print_eigenvalues(sum_tensor_products_AC, 'sum_tensor_products_AC')
print_eigenvalues(sum_tensor_products_AD, 'sum_tensor_products_AD')

# Optional: visualize the sums (real parts) for each focus


In [ ]:
Sx1 = np.array([[0, 1], [1, 0]], dtype='d')  # single-site S^x
Sy1 = np.array([[0, -1j], [1j, 0]], dtype=complex) # single-site S^z
Sz1 = np.array([[1, 0], [0, -1]], dtype='d') 
Sx1@Sy1==1j*Sz1




In [ ]:
def modify_pauli_tensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'A' and i == 2:
        matrices[0] = -Sy1 # Replace Pauli Y on A
    return matrices[0]

    if focus == 'B' and j == 2:
        matrices[1] = -Sy1   # Replace Pauli Y on B
    return matrices[1]
    if focus == 'C' and k == 2:
        matrices[2] = -Sy1   # Replace Pauli Y on C
    return marrices[2]
    if focus == 'D' and l == 2:
        matrices[3] = -Sy1   # Replace Pauli Y on D
    return matrices[3]
    

In [ ]:
modify_pauli_tensor(2, 1, 2, 3, 'A')

In [1]:
import numpy as np
from scipy.sparse import kron, identity
from scipy.linalg import ishermitian
from scipy.sparse.linalg import eigsh
from collections import namedtuple


model_d = 2  # single-site basis size

Sx1 = np.array([[0, 1], [1, 0]], dtype='d')  # single-site S^x
Sy1 = np.array([[0, -1j], [1j, 0]], dtype=complex) # single-site S^z
Sz1 = np.array([[1, 0], [0, -1]], dtype='d')  # single-site S^z
Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+
a1=np.array([1,0], dtype='d')
a2=np.array([0,1], dtype='d')
pauli_matrices = [np.eye(2), Sx1, Sy1, Sz1]
psi=np.kron(a2,np.kron(a2,np.kron(a1,a1)))/(2*np.sqrt(3))+ np.kron(a1,np.kron(a1,np.kron(a2,a2)))/(2*np.sqrt(3)) + np.kron(a1,np.kron(a2,np.kron(a2,a1)))/(2*np.sqrt(3))  +np.kron(a2,np.kron(a1,np.kron(a1,a2)))/(2*np.sqrt(3)) - np.kron(a2,np.kron(a1,np.kron(a2,a1)))/(np.sqrt(3)) - np.kron(a1,np.kron(a2,np.kron(a1,a2)))/(np.sqrt(3))
 
def partial_trace(rho, dims, trace_sites):
    """
    Takes the partial trace over the subsystems defined in 'trace_sites'.
    rho: a matrix (density matrix)
    dims: a list containing the dimension of each subsystem
    trace_sites: a list of indices of the subsystems to be traced out
    (We assume that each subsystem is square)                 
    """
   
    

# Function to modify tensor product based on focus (A, B, C, D)
def modify_pauli_tensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'A' :
        if i == 2:
            matrices[0] = -Sy1 # Replace Pauli Y on A
    elif focus == 'B':
        if j == 2:
            matrices[1] = -Sy1   # Replace Pauli Y on B
    elif focus == 'C':
        if k == 2:
            matrices[2] = -Sy1   # Replace Pauli Y on C
    elif focus == 'D' :
        if l == 2:
            matrices[3] = -Sy1   # Replace Pauli Y on D
    
    return np.kron(np.kron(np.kron(matrices[0], matrices[1]), matrices[2]), matrices[3])


def modify_pauli_dtensor(i, j, k, l, focus):
    matrices = [pauli_matrices[i], pauli_matrices[j], pauli_matrices[k], pauli_matrices[l]]
    
    if focus == 'AB':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if j == 2:
            matrices[1] = -Sy1   # Replace Pauli Y on B
    elif focus == 'AC':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if k == 2:
            matrices[2] = -Sy1   # Replace Pauli Y on C
    elif focus == 'AD':
        if i == 2:
            matrices[0] = -Sy1   # Replace Pauli Y on A
        if l == 2:
            matrices[3] = -Sy1   # Replace Pauli Y on D
    
    return np.kron(np.kron(np.kron(matrices[0], matrices[1]), matrices[2]), matrices[3])



def print_eigenvalues(tensor_sum, label):
    tensor_real = np.real(tensor_sum)  # Convert to real matrix (no need for .full())
    eigenvalues = np.linalg.eigvalsh(tensor_real)  # Compute eigenvalues
    print(f"Eigenvalues for {label}: {eigenvalues}")



def logarithmic_negativity(rho):
    """
    Compute the logarithmic negativity of the density matrix.
    
    Parameters:
    rho: The density matrix.
    dims: A list containing the dimension of each subsystem.
    subsystem: The index of the subsystem to be partially transposed.
    
    Returns:
    The logarithmic negativity of the density matrix.
    """
    # Perform partial transpose
    eigenvalues = np.linalg.eigvalsh(rho)
    
    # Compute negativity (sum of absolute values of negative eigenvalues)
    negativity = np.sum(np.abs(eigenvalues[eigenvalues < 0]))
    
    # Compute logarithmic negativity
    #log_negativity = np.log(2*negativity + 1)  # Adding small value to avoid log(0)
    
    return negativity


#def logarithmic_negativity(rho, dims, subsystem):
    #rho_pt = partial_transpose(rho, dims, subsystem)
   # eigenvalues = np.linalg.eigvals(rho_pt)
   # negativity = sum(np.abs(eigenvalues) - eigenvalues) / 2
   # log_negativity = np.log(2*negativity + 1)
    
    #return log_negativity

L = 4
dims = [model_d] * L
  
dims1 =np.array(dims)
total_dims = np.prod(dims1)
sites = [0,1] 
n1=np.delete(dims, sites)
    
# Initialize tensor product sums to handle complex values
sum_tensor_products_A = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_B = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_C = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_D = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_AB = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_AC = np.zeros((2**4, 2**4), dtype='complex128')
sum_tensor_products_AD = np.zeros((2**4, 2**4), dtype='complex128')

rho = np.outer(psi, psi.conj()).reshape((total_dims, total_dims))
    
# Specify the sites of interest (0-based indexing)
rho_sites= rho 

        # Step 4: Iterate over all combinations of Pauli matrices indexed from 0 to 3
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                tensor_product_A = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'A')) * modify_pauli_tensor(i, j, k, l, 'A') 
                   
            sum_tensor_products_A += tensor_product_A
                
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                tensor_product_B = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'B')) * modify_pauli_tensor(i, j, k, l, 'B')              
                    
            sum_tensor_products_B += tensor_product_B
                    
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                tensor_product_C = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'C')) * modify_pauli_tensor(i, j, k, l, 'C') 
                    
            sum_tensor_products_C += tensor_product_C
                    
                    
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                tensor_product_D = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'D')) * modify_pauli_tensor(i, j, k, l, 'D') 
                    
            sum_tensor_products_D += tensor_product_D
                    
        
  
for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                tensor_dproduct_AB = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AB')) * modify_pauli_dtensor(i, j, k, l, 'AB') 
                tensor_dproduct_AC = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AC')) * modify_pauli_dtensor(i, j, k, l, 'AC') 
                tensor_dproduct_AD = (1/16) * np.trace(rho_sites @ modify_pauli_tensor(i, j, k, l, 'AD')) * modify_pauli_dtensor(i, j, k, l, 'AD') 
                    
    
            sum_tensor_products_AB += tensor_dproduct_AB
            sum_tensor_products_AC += tensor_dproduct_AC
            sum_tensor_products_AD += tensor_dproduct_AD
                    
    
v1=logarithmic_negativity(sum_tensor_products_A)
v2=logarithmic_negativity(sum_tensor_products_B)
v3=logarithmic_negativity(sum_tensor_products_C)
v4=logarithmic_negativity(sum_tensor_products_D)
v5=logarithmic_negativity(sum_tensor_products_AB)
v6=logarithmic_negativity(sum_tensor_products_AC)
v7=logarithmic_negativity(sum_tensor_products_AD)
v=(v1*v2*v3*v4*v5*v6*v7)**(1/7)
def is_symmetric(matrix, tol=1e-8):
    return np.allclose(matrix, matrix.conjugate().T, atol=tol)

 
print(ishermitian(rho))  # Should return True
print(ishermitian(rho_sites))
print(is_symmetric(rho_sites))
#print("Logarithmic Negativity for sites {}:".format(n1), values)
# Function to compute and print eigenvalues

# Compute and print eigenvalues for each tensor product sum
print_eigenvalues(sum_tensor_products_A, 'sum_tensor_products_A')
print_eigenvalues(sum_tensor_products_B, 'sum_tensor_products_B')
print_eigenvalues(sum_tensor_products_C, 'sum_tensor_products_C')
print_eigenvalues(sum_tensor_products_D, 'sum_tensor_products_D')
print_eigenvalues(sum_tensor_products_AB, 'sum_tensor_products_AB')
print_eigenvalues(sum_tensor_products_AC, 'sum_tensor_products_AC')
print_eigenvalues(sum_tensor_products_AD, 'sum_tensor_products_AD')
print('Negativity w.r.t. A :', v1)
print('Negativity w.r.t. B :', v2)
print('Negativity w.r.t. C :', v3)
print('Negativity w.r.t. D :', v4)
print('Negativity w.r.t. AB :', v5)
print('Negativity w.r.t. AC :', v6)
print('Negativity w.r.t. AD :', v7)
print('Negativity :', v)
    #log_neg = logarithmic_negativity(rho_sites, dims, list(range(len(dims)-len(sites))))
   # print("Logarithmic Negativity for sites {}:".format(sites), log_neg)


True
True
True
Eigenvalues for sum_tensor_products_A: [-2.50000000e-01 -2.50000000e-01 -4.92636366e-17 -1.38777878e-17
 -1.38777878e-17 -1.26514480e-17 -4.39513190e-18  9.44254560e-19
  4.99133515e-18  1.07584841e-17  1.38777878e-17  1.38777878e-17
  2.34321668e-17  2.37191726e-17  2.50000000e-01  2.50000000e-01]
Eigenvalues for sum_tensor_products_B: [-2.50000000e-01 -2.50000000e-01 -4.92636366e-17 -1.38777878e-17
 -1.38777878e-17 -1.26514480e-17 -4.39513190e-18  9.44254560e-19
  4.99133515e-18  1.07584841e-17  1.38777878e-17  1.38777878e-17
  2.34321668e-17  2.37191726e-17  2.50000000e-01  2.50000000e-01]
Eigenvalues for sum_tensor_products_C: [-2.50000000e-01 -2.50000000e-01 -4.92636366e-17 -1.38777878e-17
 -1.38777878e-17 -1.26514480e-17 -4.39513190e-18  9.44254560e-19
  4.99133515e-18  1.07584841e-17  1.38777878e-17  1.38777878e-17
  2.34321668e-17  2.37191726e-17  2.50000000e-01  2.50000000e-01]
Eigenvalues for sum_tensor_products_D: [-2.50000000e-01 -2.50000000e-01 -4.92636366e-

In [2]:
np.kron(a1,a2)

array([0., 1., 0., 0.])

In [3]:
import numpy as np

# Define the state `psi` from your code (normalized)
a1 = np.array([1,0], dtype='d')
a2 = np.array([0,1], dtype='d')

psi = (np.kron(a2,np.kron(a2,np.kron(a1,a1)))/(2*np.sqrt(3)) + 
       np.kron(a1,np.kron(a1,np.kron(a2,a2)))/(2*np.sqrt(3)) + 
       np.kron(a1,np.kron(a2,np.kron(a2,a1)))/(2*np.sqrt(3)) +
       np.kron(a2,np.kron(a1,np.kron(a1,a2)))/(2*np.sqrt(3)) - 
       np.kron(a2,np.kron(a1,np.kron(a2,a1)))/(np.sqrt(3)) - 
       np.kron(a1,np.kron(a2,np.kron(a1,a2)))/(np.sqrt(3)))

# Normalize the state
psi_norm = psi / np.linalg.norm(psi)

# Swap two qubits, say the first and second, and check sign
def swap_qubits(state, q1, q2, num_qubits):
    """
    Swaps qubits q1 and q2 in a multi-qubit state.
    """
    swapped_state = np.reshape(state, [2]*num_qubits)
    indices = list(range(num_qubits))
    indices[q1], indices[q2] = indices[q2], indices[q1]
    return np.transpose(swapped_state, axes=indices).flatten()

# Swap qubit 1 and qubit 2 in `psi` and compare signs
psi_swapped = swap_qubits(psi_norm, 0, 1, 4)
sign_check = np.allclose(psi_norm, -psi_swapped)  # Should be True for anti-symmetric state

print("Anti-symmetric under swap:", sign_check)


Anti-symmetric under swap: False


In [4]:
# Pauli matrices
Sx = np.array([[0, 1], [1, 0]])
Sy = np.array([[0, -1j], [1j, 0]])
Sz = np.array([[1, 0], [0, -1]])

# Construct the total spin operators for a 4-qubit system
def total_spin_operator(S, num_qubits, qubit):
    """Returns the total spin operator S acting on the specified qubit."""
    op_list = [np.eye(2)] * num_qubits
    op_list[qubit] = S
    return np.kron(op_list[0], np.kron(op_list[1], np.kron(op_list[2], op_list[3])))

# Compute <psi|S^2|psi> (expectation value of total spin squared)
S_total = sum([total_spin_operator(Sx, 4, i)**2 + 
               total_spin_operator(Sy, 4, i)**2 + 
               total_spin_operator(Sz, 4, i)**2 for i in range(4)])

expectation_value = np.vdot(psi_norm, S_total @ psi_norm)
print("Expectation value of total spin squared:", expectation_value.real)


Expectation value of total spin squared: 4.000000000000001


In [5]:
singlet_12 = (np.kron(a1,a2) - np.kron(a2,a1)) / np.sqrt(2)  # Singlet for first two qubits
singlet_34 = (np.kron(a1,a2) - np.kron(a2,a1)) / np.sqrt(2)  # Singlet for third and fourth qubits

# Tensor product to create a full 4-qubit singlet-like state
singlet_full = np.kron(singlet_12, singlet_34)

# Inner product with `psi`
overlap = np.vdot(psi_norm, singlet_full)
print("Overlap with reference singlet state:", np.abs(overlap))


Overlap with reference singlet state: 0.8660254037844386
